<a href="https://colab.research.google.com/github/codethechangehmc/red-cross/blob/main/zensvi_imgdwnld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing zensvi

In [2]:
pip install zensvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of timezonefinder to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of timezonefinder to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of transformers to determine which version is compatible 

To import images from the dataset, we specify the user who uploaded the images on Mapillary (benjidad) and make an API call that retrieves the IDs of all the images.

In [4]:
from zensvi.download import MLYDownloader
import requests
import csv

# ---------------- CONFIG ----------------
ACCESS_TOKEN = "PASTE API KEY"
USERNAME = "benjidad"
MAX_IMAGES = 100          # change this to fetch more/less
CSV_FILE = "mapillary_coords.csv"
# ---------------------------------------

url = f"https://graph.mapillary.com/images?access_token={ACCESS_TOKEN}&creator_username={USERNAME}&fields=id,computed_geometry&limit=100"

coords = []
count = 0

while url and count < MAX_IMAGES:
    r = requests.get(url)
    data = r.json()

    for img in data.get("data", []):
        geom = img.get("computed_geometry", {}).get("coordinates")
        if geom:
            lon, lat = geom
            coords.append([lat, lon])
            count += 1
            if count >= MAX_IMAGES:
                break

    # follow pagination
    url = data.get("paging", {}).get("next")

# save CSV
with open(CSV_FILE, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["latitude", "longitude"])
    writer.writerows(coords)

print(f"✅ Saved {len(coords)} points to {CSV_FILE}")

✅ Saved 100 points to mapillary_coords.csv


Download CSV file produced onto PC:

In [11]:
from google.colab import files
files.download("mapillary_coords.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Using CSV file and Zensvi to get images:

In [25]:
# with a csv file with lat and lon:
downloader = MLYDownloader(mly_api_key=ACCESS_TOKEN)
downloader.download_svi("/content/images", input_csv_file="/content/mapillary_coords.csv", buffer = 1, update_pids=False)

Getting pids...


Loading cache files: 0it [00:00, ?it/s]


[Vector Tiles API] Fetching 2 tiles for images ...


Filtering data: 100%|██████████| 223014/223014 [06:49<00:00, 544.42it/s]


The panorama IDs have been saved to /content/images/mly_pids.csv
The cache directory for tiles has been deleted


Getting urls by batch size 80: 100%|██████████| 1/1 [00:08<00:00,  8.50s/it]

The cache directory has been deleted


In [21]:
import os

directory = "images/mly_svi/batch_1"
num_files = len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])
print(f"Number of files: {num_files}")

Number of files: 80


In [24]:
import shutil

folder = "/content/images"  # change to your folder path
shutil.rmtree(folder)  # deletes folder + all files/subfolders
print("Folder deleted!")

Folder deleted!
